# Come inizializzare lo stato?

In [1]:
from typing import Iterable, Union

import quimb.tensor as qtn

import pennylane as qml

from pennylane.wires import Wires

import numpy as np


def create_initial_state(
    wires: Union[qml.wires.Wires, Iterable],
    prep_operation: qml.operation.StatePrep = None,
    # like: str = "numpy",
):
    r"""
    Returns an initial state, defaulting to :math:`\ket{0}` if no state-prep operator is provided.

    Args:
        wires (Union[Wires, Iterable]): The wires to be present in the initial state
        prep_operation (Optional[StatePrep]): An operation to prepare the initial state
        like (Optional[str]): The machine learning interface used to create the initial state.
            Defaults to numpy

    Returns:
        array: The initial state of a circuit
    """
    if prep_operation:
        raise qml.DeviceError("Device does not support prep.")

    return qtn.MPS_computational_state(
        "0" * max(1, len(wires)), tags=[str(l) for l in wires.labels]
    )

/home/pietropaolo.frisoni/anaconda3/envs/PennyLane/lib/python3.9/site-packages/cotengra/hyperoptimizers/hyper.py:34: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


In [2]:
wires = Wires([0])

create_initial_state(wires)

MatrixProductState(tensors=1, indices=2, L=1, max_bond=2)

In [3]:
Wires([0])

<Wires = [0]>

In [4]:
dtype = np.complex128

In [5]:
dtype.__name__

'complex128'

## Learning from `LightningQubit`

In [6]:
from pennylane_lightning.lightning_qubit import LightningQubit

In [7]:
obj = LightningQubit(wires=[0, 1, 2])

In [8]:
obj._statevector

In [9]:
obj._statevector.state_vector

In [10]:
obj._statevector.state

array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])

## Testing `LightningTensor` 

In [11]:
from pennylane.wires import Wires

wires = Wires([0, 1, 2])

In [12]:
from pennylane_lightning.lightning_tensor import LightningTensor

In [13]:
obj = LightningTensor(wires=wires, c_dtype=np.complex128)

## `quimb` 

In [14]:
obj._statetensor.mps

MatrixProductState(tensors=3, indices=5, L=3, max_bond=2)

In [15]:
obj._statetensor.state

array([[1.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j],
       [0.+0.j]])

In [16]:
print(obj.wires)

<Wires = [0, 1, 2]>


In [17]:
config = obj.preprocess()

In [18]:
config.device_options

{'backend': 'quimb',
 'method': 'mps',
 'c_dtype': numpy.complex128,
 'max_bond_dim': None}

In [19]:
qtn.Circuit(N=len(wires), tags=[str(l) for l in wires.labels]).to_dense()

[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]

In [20]:
qc = qtn.CircuitMPS(N=len(wires), tags=[str(l) for l in wires.labels])

gates = [
    ("H", 0),
    ("H", 1),
    ("CNOT", 1, 2),
    ("CNOT", 0, 2),
    ("H", 0),
    ("H", 1),
    ("H", 2),
]

print(qc.psi.to_dense())

qc.apply_gates(gates)

print(qc.psi.to_dense())

[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
[[ 7.07106781e-01+0.j]
 [-5.26737602e-17+0.j]
 [ 7.00186921e-17+0.j]
 [ 6.00801752e-17+0.j]
 [ 6.00801752e-17+0.j]
 [ 7.00186921e-17+0.j]
 [-5.26737602e-17+0.j]
 [ 7.07106781e-01+0.j]]


In [21]:
qc.psi.max_bond()

2

In [28]:
qtn.MPS_computational_state("0" * max(1, len(wires)))

MatrixProductState(tensors=3, indices=5, L=3, max_bond=2)

In [29]:
p = qtn.MPS_rand_state(L=20, bond_dim=50)
print(f"Site tags: '{p.site_tag_id}', site inds: '{p.site_ind_id}'")

Site tags: 'I{}', site inds: 'k{}'


In [31]:
p.max_bond()

50